In [1]:
# from libs.alibaba.visitor import Visitor
from libs.alibaba.alibaba import Alibaba
from libs.json import JSON
from openpyxl import load_workbook
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq
from libs.CeleryTasks import tasks
import html
import time
import re
import os
import traceback
import types
import pendulum

In [2]:
market = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']

. storage markets.json
.\storage\markets.json was deserialized!


In [3]:
class Product_Ranking:
    api = 'https://hz-productposting.alibaba.com/product/ranksearch/rankSearch.htm'
    browser = None
    
    def __init__(self, market, headless=True, browser=None):
        self.market = market
        self.lid = market['lid']
        self.lpwd = market['lpwd']

        self.alibaba = None
        self.browser = browser
        self.headless = headless
        
    def login(self):
        self.alibaba = Alibaba(self.lid, self.lpwd, headless=self.headless, browser=self.browser)
        self.alibaba.login()
        self.browser = self.alibaba.browser
        
    def find_product_rank(self, kw, product_id):
        result = None
        input = self.browser.find_element_by_css_selector('#queryString')
        input.send_keys(Keys.CONTROL, 'a')
        input.send_keys(kw)
        input.send_keys(Keys.RETURN)

        css_table = '#rank-searech-table'
        table = WebDriverWait(self.browser, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, css_table))
        )
        for tr in table.find_elements_by_css_selector('tbody tr'):
            if '无匹配结果' in tr.text:
                return None
            a = tr.find_element_by_css_selector('td.products a')
            if product_id in a.get_attribute('href'):
                a = tr.find_element_by_css_selector('td.ranking a')
                url = a.get_attribute('href')
                text = a.text
                page = re.search('约第(\d+)页，第(\d+)位', text).group(1)
                pos = re.search('约第(\d+)页，第(\d+)位', text).group(2)
                result = {'keyword':kw, 'url':url, 'page':page, 'position':pos}
                break
        return result
    
    def fill_excel_file(self, file):
        wb = load_workbook(file)
        sheet = wb.worksheets[0]
        pid = str(sheet.cell(row=1,column=5).value)
        dt_cell = sheet.cell(row=2, column=5)
        dt_cell.value = pendulum.now().to_datetime_string()
        for i in range(7, 9999):
            cell = sheet.cell(row=i,column=4)
            if cell.value is None:
                break
            kw = ' '.join(cell.value.strip().split('\xa0'))
            print(sheet.cell(row=i,column=1).value, end='\t')
            print(sheet.cell(row=i,column=2).value, end='\t')
            print(sheet.cell(row=i,column=3).value, end='\t')
            print(kw, end='\t')
            ranking = self.find_product_rank(kw, pid)
            if ranking:
                result = ranking['page'] + '-' + ranking['position']
            else:
                result = 'X'
            cell = sheet.cell(row=i,column=5)
            cell.value = result
            print(result)
        wb.save(file)
        print('done!')
        
    def find_keyword_ranking(self, kw):

        result = []
        input = self.browser.find_element_by_css_selector('#queryString')
        input.send_keys(Keys.CONTROL, 'a')
        input.send_keys(kw)
        input.send_keys(Keys.RETURN)

        css_table = '#rank-searech-table'
        table = WebDriverWait(self.browser, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, css_table))
        )

        for tr in table.find_elements_by_css_selector('tbody tr'):
            if '无匹配结果' in tr.text:
                break
            a = tr.find_element_by_css_selector('td.products a')
            url = a.get_attribute('href')
            ali_id = re.search('\?id=(\d+)', url).group(1)
            a = tr.find_element_by_css_selector('td.ranking a')
            text = a.text
            page = re.search('约第(\d+)页，第(\d+)位', text).group(1)
            pos = re.search('约第(\d+)页，第(\d+)位', text).group(2)

            result.append({'keyword':kw, 'url':url, 'ali_id': ali_id, 'page':page, 'position': pos})
            break
        return result

In [17]:
def fill_excel_file(self, file):
    wb = load_workbook(file)
    sheet = wb.worksheets[0]
    pid = str(sheet.cell(row=1,column=5).value)
    dt_cell = sheet.cell(row=2, column=5)
    dt_cell.value = pendulum.now().to_datetime_string()
    for i in range(7, 9999):
        cell = sheet.cell(row=i,column=4)
        if cell.value is None:
            break
        kw = ' '.join(cell.value.strip().split('\xa0'))
        print(sheet.cell(row=i,column=1).value, end='\t')
        print(sheet.cell(row=i,column=2).value, end='\t')
        print(sheet.cell(row=i,column=3).value, end='\t')
        print(kw, end='\t')
        ranking = self.find_product_rank(kw, pid)
        if ranking:
            result = ranking['page'] + '-' + ranking['position']
        else:
            result = 'X'
        cell = sheet.cell(row=i,column=5)
        cell.value = result
        print(result)
    wb.save(file)
    print('done!')

pr.fill_excel_file = types.MethodType(fill_excel_file, pr)

In [ ]:
pr.find_keyword_ranking = types.MethodType(find_keyword_ranking, pr)

In [4]:
pr = Product_Ranking(market, headless=False)

In [5]:
pr.login()

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy


In [6]:
pr.browser.get(pr.api)

In [ ]:
#pr.find_keyword_ranking('mink eyelash')

In [8]:
# pr.find_product_rank(kw, pid)

In [7]:
files = []
for i in range(50):
    files.append('')

files[0] = 'product_ranking_mink_eyelash.xlsx'
files[1] = 'product ranking magnetic eyelash.xlsx'
files[2] = 'product_ranking_Easy_Fan_Lashes.xlsx'
files[3] = 'product_ranking_mink_eyelash_extension.xlsx'
files[4] = 'product_ranking_individual_eyelash_extension.xlsx'

In [8]:
file = files[0]
pr.fill_excel_file(file)

517	367	4248	mink eyelash	3-16
484	362	2946	3d mink eyelash	2-36
398	242	1879	mink lash	X
91	15	1163	mink eyelash vendor	5-25
340	122	1093	wholesale mink eyelash	3-1
379	255	1057	3d mink lash	X
441	194	949	private label mink eyelash	3-13
237	26	939	eyelash mink	X
183	26	782	3d mink eyelash private label	2-32
165	6	726	mink 3d eyelash	3-29
223	32	556	mink eyelash 3d	X
285	48	447	real mink eyelash	X
43	4	411	cruelty free 3d mink eyelash	1-24
135	13	397	eyelash mink	X
377	101	327	mink strip eyelash	X
9	0	304	mink eyelash with box	X
152	8	277	100% mink eyelash	X
340	75	254	mink fur eyelash	X
144	12	252	wholesale 3d mink eyelash	2-32
76	1	244	eyelash mink private label	2-35
52	1	232	free sample mink eyelash	X
103	5	218	bulk mink lash	X
141	6	209	cruelty free mink eyelash	1-28
7	0	199	mink eyelash wholsale	X
140	6	199	premium mink eyelash	X
128	9	192	mink eyelash wholesale	3-1
181	15	176	private_label mink lash	X
21	1	164	wholesale mink strip lash	X
104	2	155	natural mink eyelash	X
95	4	143	

In [ ]:
product_ranking_overview

In [65]:
%store -r sorted_kws

In [ ]:
products = pr.alibaba.get_posted_product_info(999)

In [94]:
ps = {}
for p in products:
    ps[p['ali_id']] = p

In [ ]:
file = 'products ranking overview.xlsx'
wb = load_workbook(file)
sheet = wb.worksheets[0]
for i in range(1,999):
    kw = sheet.cell(row=i,column=2).value
    if kw is None:
        break
    print(sheet.cell(row=i,column=1).value, kw, end="\t")
    while True:
        try:
            result = pr.find_keyword_ranking(kw)
            break
        except TimeoutException:
            pr.browser.get(pr.api)
            continue
    if result:
        fp = result[0]
        p = ps[fp['ali_id']]
        sheet.cell(row=i, column=3).value = fp['page']
        sheet.cell(row=i, column=4).value = fp['position']
        sheet.cell(row=i, column=5).value = fp['ali_id']
        sheet.cell(row=i, column=6).value = p['pid']
        sheet.cell(row=i, column=7).value = p['title']
    else:
        sheet.cell(row=i, column=3).value = ' '
        sheet.cell(row=i, column=4).value = ' '
        sheet.cell(row=i, column=5).value = ' '
        sheet.cell(row=i, column=6).value = ' '
        sheet.cell(row=i, column=7).value = ' '
        
    print(fp['page'], fp['position'], fp['ali_id'], p['pid'], p['title'])
#     break
wb.save(file)